In [3]:
from haystack_integrations.components.generators.ollama import OllamaGenerator

from haystack import Pipeline, Document
from haystack.components.retrievers.in_memory import InMemoryBM25Retriever
from haystack.components.builders.prompt_builder import PromptBuilder
from haystack.document_stores.in_memory import InMemoryDocumentStore

import logging

logging.basicConfig(format="%(levelname)s - %(name)s -  %(message)s", level=logging.WARNING, 
                    filemode='w', filename='app.log')
logging.getLogger("haystack").setLevel(logging.DEBUG)

query = "What do I like?"

template = """
Given the following information, answer the question.

Context: 
{% for document in documents %}
    {{ document.content }}
{% endfor %}

Question: {{ query }}?
"""


docstore = InMemoryDocumentStore()
docstore.write_documents([Document(content="I really like summer"),
                          Document(content="My favorite sport is soccer"),
                          Document(content="I don't like reading sci-fi books"),
                          Document(content="I don't like crowded places"),])

generator = OllamaGenerator(model="llama3.2",
                            url = "http://localhost:11434",
                            generation_kwargs={
                              "num_predict": 100,
                              "temperature": 0.9,
                              })

pipe = Pipeline()
pipe.add_component("retriever", InMemoryBM25Retriever(document_store=docstore))
pipe.add_component("prompt_builder", PromptBuilder(template=template))
pipe.add_component("llm", generator)
pipe.connect("retriever", "prompt_builder.documents")
pipe.connect("prompt_builder", "llm")

result = pipe.run({"prompt_builder": {"query": query},
									"retriever": {"query": query}})

print(result)


{'llm': {'replies': ["Based on the context you provided, here are some things that you seem to like:\n\n1. Summer\n2. Soccer (your favorite sport)\n3. Possibly other outdoor or recreational activities (not specified, but implied by your liking of summer and not crowded places)\n\nNote that there's one thing you don't like in the list: reading sci-fi books."], 'meta': [{'model': 'llama3.2', 'created_at': '2025-02-28T18:39:15.8978676Z', 'done': True, 'done_reason': 'stop', 'total_duration': 1151124700, 'load_duration': 14097000, 'prompt_eval_count': 75, 'prompt_eval_duration': 1000000, 'eval_count': 73, 'eval_duration': 1134000000, 'context': [128006, 9125, 128007, 271, 38766, 1303, 33025, 2696, 25, 6790, 220, 2366, 18, 271, 128009, 128006, 882, 128007, 1432, 22818, 279, 2768, 2038, 11, 4320, 279, 3488, 382, 2014, 25, 4815, 262, 358, 2216, 1093, 7474, 271, 262, 358, 1541, 956, 1093, 39313, 7634, 271, 262, 358, 1541, 956, 1093, 5403, 39074, 37731, 6603, 271, 262, 3092, 7075, 10775, 374, 2